In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers, utils
import numpy as np
import pandas as pd

In [3]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [4]:
seed = 7
np.random.seed(seed)

In [5]:
dataframe = pd.read_csv("sonar.csv", header=None)
dataset = dataframe.values
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]

In [30]:
# baseline model
def create_baseline():
    model = models.Sequential()
    model.add(layers.Dense(60, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))          
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
    return model

In [31]:
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X,Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Results: 81.23% (6.98%)


# Re-run model with data preparation

In [12]:
np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Standardized: 86.97% (4.41%)


#  Smaller Model

In [16]:
def create_smaller():
    model = models.Sequential()
    model.add(layers.Dense(30, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))          
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
    return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))



Smaller: 82.68% (6.11%)


#  Large Model

In [33]:
# larger model
def create_larger():
    model = models.Sequential()
    model.add(layers.Dense(60, activation='relu'))
    model.add(layers.Dense(30, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))          
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
    return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Larger: 85.52% (6.15%)


# Overfitting

In [35]:
def create_larger():
    model = models.Sequential()
    model.add(layers.Dense(60, activation='relu'))
    model.add(layers.Dense(50, activation='relu'))
    model.add(layers.Dense(40, activation='relu'))
    model.add(layers.Dense(30, activation='relu'))
    model.add(layers.Dense(20, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))          
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
    return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, epochs=300, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X,Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Larger: 81.68% (7.22%)


# Using Keras Functional  API

In [36]:
def create_model():
    input = keras.Input(shape=(10,))
    x = layers.Dense(60, activation = 'relu')(x)
    x = layers.Dense(60, activation = 'relu')(x)
    output = layers.Dense(1,activation='sigmoid')(x)
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
    return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, epochs=300, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X,Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Larger: 83.63% (6.51%)


# Using Keras Model Subclassing

In [42]:
import tensorflow.keras as k
class MyModel(k.Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.dense1 = k.layers.Dense(60, activation='relu')
    self.dense2 = k.layers.Dense(1, activation='sigmoid')

  def call(self, inputs):
    x = self.dense1(inputs)
    return self.dense2(X)

model = MyModel()

In [43]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])